<h1>Важность признаков для выбранного алгоритма классификации</h1>

Загрузим необходимы библиотеки

In [1]:
import pandas as pd
import gc
import warnings
from sklearn import model_selection
import catboost as cb

gc.enable()
warnings.filterwarnings("ignore")

Загрузим обработанные данные

In [2]:
data = pd.read_csv("data/scoring_processed.csv")

cat_features = ["RealEstateLoansOrLines", "GroupAge"]
data = pd.get_dummies(data, columns=cat_features, drop_first=True)

cols = data.columns[1:].tolist()

Разделим на трейн и валидационную части и соберём пулы для `CatBoostClassifier`

In [3]:
train, valid, _, _ = model_selection.train_test_split(
    data,
    data["SeriousDlqin2yrs"],
    test_size=0.25,
    stratify=data["SeriousDlqin2yrs"],
    shuffle=True,
    random_state=5432
)

In [4]:
train_pool = cb.Pool(
    train[cols],
    train["SeriousDlqin2yrs"],
    cat_features=None
)

valid_pool = cb.Pool(
    valid[cols],
    valid["SeriousDlqin2yrs"],
    cat_features=None
)

Обучим классификатор

In [5]:
clf = cb.CatBoostClassifier(random_seed=5432, early_stopping_rounds=200, eval_metric="F1", verbose=200)
clf.fit(train_pool, eval_set=valid_pool)

Learning rate set to 0.097828
0:	learn: 0.6361548	test: 0.6389653	best: 0.6389653 (0)	total: 168ms	remaining: 2m 48s
200:	learn: 0.7333673	test: 0.7219662	best: 0.7219662 (199)	total: 1.75s	remaining: 6.98s
400:	learn: 0.7426018	test: 0.7251850	best: 0.7251850 (397)	total: 3.47s	remaining: 5.19s
600:	learn: 0.7507548	test: 0.7242610	best: 0.7260692 (555)	total: 5.2s	remaining: 3.45s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7260692464
bestIteration = 555

Shrink model to first 556 iterations.


Посмотрим на полученную важность признаков

In [6]:
pd.DataFrame({
    "col": clf.feature_names_,
    "fi": clf.feature_importances_
}).sort_values(by="fi", ascending=False)

,col,fi
5,NumberOfTimes90DaysLate,22.500234
0,RevolvingUtilizationOfUnsecuredLines,18.021585
6,NumberOfTime60-89DaysPastDueNotWorse,13.451813
1,NumberOfTime30-59DaysPastDueNotWorse,10.732799
2,DebtRatio,10.102843
3,MonthlyIncome,8.402733
4,NumberOfOpenCreditLinesAndLoans,6.819184
7,NumberOfDependents,5.408075
14,GroupAge_e,1.458092
13,GroupAge_d,1.035770


Получаем, что самым важным признаком является `NumberOfTimes90DaysLate`, а признаком, который не вносит вклад в предсказательную способность модели, является `RealEstateLoansOrLines_E`